# Importando Bibliotecas

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Inicializando DataFrames

In [4]:
df = pd.read_csv('datasets/users_behavior.csv')

In [8]:
print(df.head())
df.info()

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


# Desenvolvimento do Modelo

### Criando as respectivas features e targets para treinamento e validação

In [9]:
feature = df.drop('is_ultra', axis=1)
target = df['is_ultra']

feature_train, feature_valid, target_train, target_valid = train_test_split(feature, target, test_size=0.25, random_state=12345)

## Treinando os modelos

### Regressão Logística

In [41]:
logistic_model = LogisticRegression(random_state=12345, solver='liblinear')
logistic_model.fit(feature_train, target_train)

logistic_prediction = logistic_model.predict(feature_valid)
logistic_accuracy = accuracy_score(target_valid, logistic_prediction)
print('Acurácia pelo modelo de regressão logística:', logistic_accuracy)

Acurácia pelo modelo de regressão logística: 0.7027363184079602


<span style='color:red'> No modelo de regressão logística obtivemos o valor de 0.70 indicando que 70% dos valores estavam corretos, espera-se que este seja o pior modelo que está inclusive abaixo do limite de 0.75, dito isso foi fácil de implementar e o resultado foi gerado rapidamente.

### Árvore de Decisão

In [38]:
best_accuracy = 0.75
best_tree_depth = 0
for depth in range(1, 10):
    decision_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    decision_tree_model.fit(feature_train, target_train)

    decision_tree_prediction = decision_tree_model.predict(feature_valid)
    current_accuracy = accuracy_score(target_valid, decision_tree_prediction)
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_tree_depth = depth
decision_tree_accuracy = best_accuracy
print(f'Acurácia pelo modelo de árvore de decisão: {decision_tree_accuracy} com depth de {best_tree_depth}')

Acurácia pelo modelo de árvore de decisão: 0.7898009950248757 com depth de 7


<span style='color:red'> No modelo de árvore de decisão o resultado foi de 0.79, ou seja, 79% dos valores estavam corretos, acima do limite, precisa de mais tempo para encontrar o depth ideal mas ainda foi relativamente rapido de gerar resultado. OBS: o depth foi testado com valores até 50 mas 7 sempre foi o depth com melhor ácuracia, portanto reduzi para 10 pra poder rodar o código mais rapido.

### Floresta Aleatória

In [39]:
best_accuracy = 0.75
best_forest_depth = 0
best_n_estimators = 1
for depth in range(1, 10):
    for n in range(1, 20):
        random_forest_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=n)
        random_forest_model.fit(feature_train, target_train)

        random_forest_prediction = random_forest_model.predict(feature_valid)
        current_accuracy = accuracy_score(target_valid, random_forest_prediction)
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_forest_depth = depth
            best_n_estimators = n
random_forest_accuracy = best_accuracy
print(f'Acurácia pelo modelo de floresta aleatória: {random_forest_accuracy} com depth de {best_forest_depth} e n_estimators de {best_n_estimators}')

Acurácia pelo modelo de floresta aleatória: 0.8146766169154229 com depth de 9 e n_estimators de 19


<span style='color:red'> No modelo de floresta aleatória obtivemos 81% de acurácia, o melhor resultado entre os 3 modelos testados, este modelo já precisou de mais tempo para encontrar não só o depth ideal mas o n_estimator ideal também, que gerou mais custo computacional. OBS: Os valores de depth iam até 20 e o n_estimators até 50 originalmente mas o melhor valor encontrado foi depth 9 com n_estimators 19, com isso eu reduzi os valores para que rodasse mais rápido

## Comparação Final

In [40]:
print('Acurácia pelo modelo de regressão logística:', logistic_accuracy)
print(f'Acurácia pelo modelo de árvore de decisão: {decision_tree_accuracy} com depth de {best_tree_depth}')
print(f'Acurácia pelo modelo de floresta aleatória: {random_forest_accuracy} com depth de {best_forest_depth} e n_estimators de {best_n_estimators}')

Acurácia pelo modelo de regressão logística: 0.7027363184079602
Acurácia pelo modelo de árvore de decisão: 0.7898009950248757 com depth de 7
Acurácia pelo modelo de floresta aleatória: 0.8146766169154229 com depth de 9 e n_estimators de 19


# Prova Real dos Modelos

In [44]:
df['is_ultra'].value_counts(normalize=True)

is_ultra
0    0.693528
1    0.306472
Name: proportion, dtype: float64

<span style='color:red'> 30.65% dos usuários são ultra, significa que se classificarmos todos os usuários como não ultra, teriamos 69.35% de acurácia, o que significa que todos os modelos são melhores do que se classificarmos ao acaso